In [1]:
import pandas as pd
import re
import numpy as np
import functions
import datetime

In [2]:
cur = functions.connectToPs()

Connecting to Postgree...

<connection object at 0x00000172441E1598; dsn: 'user=oxbigdatabi password=xxx dbname=bigdatabi host=bigdatabi.ckq5duwtue6h.eu-central-1.rds.amazonaws.com', closed: 0>


### Tabla Detalle

In [13]:
prueba = functions.detalle(cur)

In [15]:
prueba.to_excel('./output/detalle_cartera_20200127.xlsx',encoding='CP1252',index=False)

# Base

In [3]:
df = functions.base(cur)

In [4]:
print('Filas Cartera:', len(df))
print('Total ventas:', len(df[df['isSold']==1]))
print('Total cancelaciones:', len(df[df['isCancelled']==1]))
print('Total assets:', len(df[df['isAsset']==1]))
print('Total bajas:', len(df[df['isBaja']==1]))
print('Total assets activos:', len(df[df['isActive']==1]))

Filas Cartera: 1274
Total ventas: 1274
Total cancelaciones: 44
Total assets: 1142
Total bajas: 218
Total assets activos: 924


In [6]:
df.to_excel('./output/df.xlsx',encoding='CP1252',index=False)

# VENTAS

In [6]:
functions.ass_vta_mes(df,tipo=4)

2018-10  2018-11  2018-12  2019-01  \
Family       product_name                                                  
X FIBER      X Fiber 100 - neba             0        0        0        1   
             X Fiber 100 - own              0        0        4        3   
             X Fiber 200 - neba             0        1        1        0   
             X Fiber 200 - own              0        0        1        0   
             X Fiber 50 - neba              0        1        1        1   
             X Fiber 50 - own               0        2        1        2   
X MUSIC      Assets                         3        2        3        4   
X PRIVACY    X Privacy 100                  0        0        0        0   
             X Privacy 200                  0        0        1        0   
             X Privacy 50                   0        5        3        0   
X PROTECTION Assets                         0        0        4        2   
X SECURITY   X Security 100                 0        0        2        2   
             X Security 200                 0        0        0        0   
             X Security Monosede 200        0        0        0        0   
X UCOM       Teamwork+PBX                   0       13       13       20   

                                      2019-02  2019-03  2019-04  2019-05  \
Family       product_name                                                  
X FIBER      X Fiber 100 - neba             2        0        1        2   
             X Fiber 100 - own              2        0        4        3   
             X Fiber 200 - neba             3        1        2        2   
             X Fiber 200 - own              0        0        0        6   
             X Fiber 50 - neba              3        0        0        0   
             X Fiber 50 - own               0        1        1        0   
X MUSIC      Assets                         7        2        0        6   
X PRIVACY    X Privacy 100                  2        0        2        0   
             X Privacy 200                  0        0        0        4   
             X Privacy 50                  10       14        2        8   
X PROTECTION Assets                         5       25       13        4   
X SECURITY   X Security 100                 0        0        0        5   
             X Security 200                 0        0        0        0   
             X Security Monosede 200        0        0        0        0   
X UCOM       Teamwork+PBX                  22       11       16       22   

                                      2019-06  2019-07  2019-08  2019-09  \
Family       product_name                                                  
X FIBER      X Fiber 100 - neba             0        1        1        1   
             X Fiber 100 - own             12        1        0       15   
             X Fiber 200 - neba             4        5        0        2   
             X Fiber 200 - own              0        2        1        2   
             X Fiber 50 - neba              0        4        1        0   
             X Fiber 50 - own               1        4        1        0   
X MUSIC      Assets                         0        0        0        5   
X PRIVACY    X Privacy 100                  3        0        0        0   
             X Privacy 200                  5        1        0        4   
             X Privacy 50                   5        5        2        5   
X PROTECTION Assets                        29       20        5       20   
X SECURITY   X Security 100                11        2        0       50   
             X Security 200                 2        3        0        3   
             X Security Monosede 200        0        0        0        0   
X UCOM       Teamwork+PBX                  65       57       12       59   

                                      2019-10  2019-11  2019-12  2020-01  
Family       product_name                                                 
X FIBER  

In [6]:
functions.cli_vta_mes(df,2)

,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01
canal_venta,,,,,,,,,,,,,,,,
DHO,2,6,7,9,23,14,7,15,34,43,12,35,40,36,34,30
IT integrator,0,1,4,3,1,1,8,3,5,4,1,7,2,2,1,1
WEB,1,4,8,1,1,7,4,7,6,4,3,9,7,4,3,5


# Altas

In [12]:
df_altas = functions.base_altas(df)

In [13]:
df_altas.head(2)

,cif,canal_venta,assets_act,rfb,deactivation_date,assets_tot,isClient,isActive,assets_dea,isBaja
0,03817373V,WEB,0,2019-11-20 00:00:01,2019-11-20 15:29:08,2,1,0,2,1
1,07477239P,DHO,0,2018-10-25 09:38:28,2019-01-21 09:13:32,1,1,0,1,1


In [13]:
df_altas.to_excel('./detalle_altas.xlsx',encoding='CP1252',index=False)

In [69]:
functions.cli_bajas_mes(df_altas,tipo=1)

,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01
canal_venta,,,,,,,,,,,,,,,,
DHO,0,0,0,1,0,2,4,4,2,3,2,3,5,4,6,6
IT integrator,0,0,0,0,0,0,0,2,1,0,0,3,0,0,0,1
WEB,0,0,1,2,0,0,2,2,2,1,3,2,5,3,1,0


In [71]:
functions.cli_altas_mes(df_altas,tipo=1)

,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01
canal_venta,,,,,,,,,,,,,,,,
DHO,2,2,9,7,19,8,8,13,29,30,19,28,35,34,24,27
IT integrator,0,1,3,3,0,1,6,2,3,2,0,5,0,1,0,0
WEB,1,2,6,3,1,5,4,6,4,4,2,7,9,4,1,4


In [89]:
functions.ass_altas_mes(df, tipo=0)

,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01
isAsset,3,11,32,37,43,55,49,37,114,111,38,101,134,137,139,101


In [96]:
functions.ass_bajas_mes(df, tipo=1)

,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01
Family,,,,,,,,,,,,,,,,
X FIBER,0,0,0,0,0,2,0,0,1,3,3,3,2,0,2,2
X MUSIC,0,0,1,1,0,1,2,2,0,0,1,1,0,0,0,0
X PRIVACY,0,0,0,0,0,0,6,4,2,2,10,2,5,0,0,0
X PROTECTION,0,0,0,2,0,0,2,3,4,0,1,2,1,13,2,6
X SECURITY,0,0,0,0,0,0,0,0,0,2,0,0,3,0,3,1
X UCOM,0,0,0,3,2,6,1,3,7,11,9,19,14,7,13,20


## Migras

In [3]:
mig_df=functions.migBase(cur)
mig_df.head(2)

,assetid,assethijo_mig,product_name,Family,cif,canal_venta,mig_date,tipo_migra
0,02i0N00000I3wptQAB,None,Teamwork+PBX,X UCOM,B98115520,DHO,2019-05-09 17:21:03,IN
1,02i0N00000KmJaoQAF,None,Teamwork+PBX,X UCOM,B78909611,DHO,2019-07-19 16:34:33,IN


In [45]:
migin, migout = functions.mig_producto_mes(mig_df)

In [46]:
migin

2018-10  2018-11  2018-12  2019-01  \
canal_venta   Family  product_name                                             
DHO           X FIBER X Fiber 100 - neba      0.0      0.0      0.0      0.0   
              X UCOM  Teamwork+PBX            0.0      0.0      0.0      0.0   
IT integrator X UCOM  Teamwork+PBX            0.0      0.0      0.0      0.0   

                                          2019-02  2019-03  2019-04  2019-05  \
canal_venta   Family  product_name                                             
DHO           X FIBER X Fiber 100 - neba      0.0      0.0      1.0      0.0   
              X UCOM  Teamwork+PBX            0.0      0.0      0.0     11.0   
IT integrator X UCOM  Teamwork+PBX            0.0      0.0      0.0      0.0   

                                          2019-06  2019-07  2019-08  2019-09  \
canal_venta   Family  product_name                                             
DHO           X FIBER X Fiber 100 - neba      0.0      0.0      0.0      0.0   
              X UCOM  Teamwork+PBX            0.0      2.0      1.0      1.0   
IT integrator X UCOM  Teamwork+PBX            1.0      0.0      0.0      0.0   

                                          2019-10  2019-11  2019-12  2020-01  
canal_venta   Family  product_name                                            
DHO           X FIBER X Fiber 100 - neba      0.0      0.0      0.0      0.0  
              X UCOM  Teamwork+PBX            3.0      1.0      0.0      1.0  
IT integrator X UCOM  Teamwork+PBX            0.0      0.0      0.0      0.0

In [47]:
migout

2018-10  2018-11  2018-12  2019-01  \
canal_venta   Family  product_name                                             
DHO           X FIBER X Fiber 200 - neba      0.0      0.0      0.0      0.0   
              X UCOM  Teamwork+PBX            0.0      0.0      0.0      0.0   
IT integrator X UCOM  Teamwork+PBX            0.0      0.0      0.0      0.0   

                                          2019-02  2019-03  2019-04  2019-05  \
canal_venta   Family  product_name                                             
DHO           X FIBER X Fiber 200 - neba      0.0      0.0      1.0      0.0   
              X UCOM  Teamwork+PBX            0.0      0.0      0.0     11.0   
IT integrator X UCOM  Teamwork+PBX            0.0      0.0      0.0      0.0   

                                          2019-06  2019-07  2019-08  2019-09  \
canal_venta   Family  product_name                                             
DHO           X FIBER X Fiber 200 - neba      0.0      0.0      0.0      0.0   
              X UCOM  Teamwork+PBX            0.0      2.0      1.0      1.0   
IT integrator X UCOM  Teamwork+PBX            1.0      0.0      0.0      0.0   

                                          2019-10  2019-11  2019-12  2020-01  
canal_venta   Family  product_name                                            
DHO           X FIBER X Fiber 200 - neba      0.0      0.0      0.0      0.0  
              X UCOM  Teamwork+PBX            3.0      1.0      0.0      1.0  
IT integrator X UCOM  Teamwork+PBX            0.0      0.0      0.0      0.0

In [32]:
def getVelocityFrom(description):
    if re.search('4G', description):
        return '4G'
    elif re.findall('[0-9]{3}|[0-9]{2}|[0-9]{1}', description):
        if len(re.findall('[0-9]{3}|[0-9]{2}|[0-9]{1}', description)[0]) == 1:
            print(re.findall('[0-9]{3}|[0-9]{2}|[0-9]{1}', description)[0])
            return re.findall('[0-9]{3}|[0-9]{2}|[0-9]{1}', description)[0] + 'Gb'
        elif re.findall('[0-9]{3}|[0-9]{2}|[0-9]{1}', description)[0] == '20':
            return 'Portable 20'
        else:
            return re.findall('[0-9]{3}|[0-9]{2}|[0-9]{1}', description)[0]
    else:
        return ''

In [33]:
getVelocityFrom('20')

'Portable 20'